In [1]:
import argparse
import roypy
import time
import queue
from sample_camera_info import print_camera_info
from roypy_sample_utils import CameraOpener, add_camera_opener_options
#from roypy_platform_utils import PlatformHelper

import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import math
import socket_sender
import _Tranform_Data
import _Feet_detection_by_center
import _RANSAC
import _Feet_detection_v2


try:
    import roypycy
except ImportError:
    print("Pico Flexx backend requirements (roypycy) not installed properly")
    raise

In [2]:
# # RANSAM

def Dis_pt2plane(pts, a, b, c, d):
    """
    Compute the distance from points to the plane
    """
    normal = math.sqrt(a*a+b*b+c*c)
    if normal == 0:
        normal = 1
    
    v = np.array([a,b,c])
    dis = abs(np.dot(pts,v.T)+d)/normal
    return dis

def get_Plane(window_corner):
    """
    Compute the equation of the plane
    """
    p1 = window_corner[0,:]
    p2 = window_corner[1,:]
    p3 = window_corner[2,:]
    
    a = ( (p2[1]-p1[1])*(p3[2]-p1[2])-(p2[2]-p1[2])*(p3[1]-p1[1]) )
    b = ( (p2[2]-p1[2])*(p3[0]-p1[0])-(p2[0]-p1[0])*(p3[2]-p1[2]) )
    c = ( (p2[0]-p1[0])*(p3[1]-p1[1])-(p2[1]-p1[1])*(p3[0]-p1[0]) )
    d = ( 0-(a*p1[0]+b*p1[1]+c*p1[2]) )
    
    return a,b,c,d



def get_inliner_num(points3D,a,b,c,d,inliner_threshold):
    """
    Compute the liner points which distance to plane < threshold
    Also get distance from points to the plane (new Depth Image which re-project depth pixels in surface plane)
    """
    inliner_num = 0
    
    dist = Dis_pt2plane(points3D,a,b,c,d)
    inliner_mask = dist < inliner_threshold
    inliner_num = np.sum(inliner_mask)
    return inliner_num, inliner_mask, dist


In [3]:
class MyListener(roypy.IDepthDataListener):
    def __init__(self, xqueue, yqueue, zqueue, grayValuequeue):
        super(MyListener, self).__init__()
        self.xqueue = xqueue
        self.yqueue = yqueue
        self.zqueue = zqueue
        self.grayValuequeue = grayValuequeue
        self.Listening = True

    def onNewData(self, data):   
        if(self.Listening):
            t_time = time.time()
            
            xvalues = []
            yvalues = []
            zvalues = []
            grayvalues = []
            
            values = roypycy.get_backend_data(data)

            xvalues = values.x
            yvalues = values.y
            zvalues = values.z
            grayvalues = values.grayValue

            xarray = np.asarray(xvalues)
            yarray = np.asarray(yvalues)
            zarray = np.asarray(zvalues)
            
            
            q_x = xarray.reshape (-1, data.width)        
            self.xqueue.put(q_x)
            q_y = yarray.reshape (-1, data.width)        
            self.yqueue.put(q_y)
            q_z = zarray.reshape (-1, data.width)        
            self.zqueue.put(q_z)
            
            q_grayvalues = grayvalues.reshape (-1, data.width)        
            self.grayValuequeue.put(q_grayvalues)
            
            #print('store time:', (time.time()-t_time))

    def paint (self, data, name):
        """Called in the main thread, with data containing one of the items that was added to the
        queue in onNewData.
        """
        cv2.namedWindow(name, cv2.WINDOW_NORMAL)
        cv2.imshow(name, data)
        cv2.waitKey(1)




def main ():
    parser = argparse.ArgumentParser (usage = __doc__)
    add_camera_opener_options (parser)
    parser.add_argument ("--seconds", type=int, default=15, help="duration to capture data")
    parser.add_argument ("--SendData", type=bool, default=False, help="SendData")
    parser.add_argument ("--Project_Type", type=int, default=0, help="0: Project_on_body, 1: Project_on_floor")
    
    timer_show = False
    
    # 測試用 setting
    _Replay = True
    if(_Replay == True):
#         options = parser.parse_args(args=['--rrf', 'upper_line_body.rrf','--seconds', '25', '--Project_Type', '0'])
        # project on body: upper_line_body.rrf
        
        options = parser.parse_args(args=['--rrf', 'feet.rrf','--seconds', '25', '--Project_Type', '1'])
#         options = parser.parse_args(args=['--rrf', 'feet_35fps_3.rrf','--seconds', '25', '--Project_Type', '2'])
        # project on floor: feet_35fps.rrf  feet_35fps_2.rrf feet0221
    else:
#         options = parser.parse_args(args=['--seconds', '30', '--Project_Type', '0'])
        options = parser.parse_args(args=['--seconds', '30', '--Project_Type', '1'])
#         options = parser.parse_args(args=['--seconds', '30', '--Project_Type', '2'])
#         options = parser.parse_args(args=['--seconds', '30', '--Project_Type', '3'])
        

    opener = CameraOpener (options)
    cam = opener.open_camera ()
    
    if(_Replay == False):
        #MODE_9_5FPS_2000 MODE_5_45FPS_500 MODE_5_35FPS_600
#         cam.setUseCase('MODE_5_45FPS_500') # body
        cam.setUseCase('MODE_5_35FPS_600') #floor

    #Print camera information
    print_camera_info (cam)
    print("isConnected", cam.isConnected())
    print("getFrameRate", cam.getFrameRate())
    print("UseCase",cam.getCurrentUseCase())

    # we will use this queue to synchronize the callback with the main
    # thread, as drawing should happen in the main thread 
    x = queue.LifoQueue()
    y = queue.LifoQueue()
    z = queue.LifoQueue()
    grayvalue = queue.LifoQueue()
    l = MyListener(x,y,z,grayvalue)
    cam.registerDataListener(l)
    cam.startCapture()
    
    # create a loop that will run for a time (default 15 seconds)
    process_event_queue (x, y, z, grayvalue, l, options.seconds, options.Project_Type, cam)
    cam.stopCapture()
    socket_sender.close_socket()
    
    cv2.destroyAllWindows()
    
    


    

In [4]:
def process_event_queue (x,y,z,grayvalue, painter, seconds, Project_Type, cam):
    SendData = True
    Record_img = False
#     Project_Type = 0  # 0: project on body, 1: project on floor, 3: calibration shift
    
    #initial
    last_inliner_num = 20000
    quad_mask = np.zeros((171,224))
    circles_image = np.zeros((171,224,3))
    num_of_frame = 0
    feet_mask = np.ones((171,224))
    window_corner = np.ones((4,3))
    
    # create a loop that will run for the given amount of time
    print("  Quit : Q\n")
    while 1 :
        t_time = time.time()
        try:
            # try to retrieve an item from the queue.
            # this will block until an item can be retrieved
            # or the timeout of 1 second is hit
            t_time = time.time()
            
            item_x = x.get(True, 0.5)
            item_y = y.get(True, 0.5)
            item_z = z.get(True, 0.5)
            points3D = np.dstack((item_x,item_y,item_z))
            item_grayvalue = grayvalue.get(True, 0.5)
            #print('queue time:', (time.time()-t_time))
        except queue.Empty:
            # this will be thrown when the timeout is hit
            print("Empty")
#             break
            continue
        else:
            if(SendData):
                stop, window_corner = _Tranform_Data.receive_data(cam, window_corner)
                if stop:
                    break
                
                # plane equation by 3 window points send from unity
                a,b,c,d = get_Plane(window_corner)
                inliner_num, plane_mask, depthImage = get_inliner_num(points3D,a,b,c,d,inliner_threshold = 0.01)
                
                #not need find plane by RANSAC => for test not need RANSAC
                Confidence_img = points3D[:,:,2] != 0 
                surface_plane, depthImg, plane_mask, best_sampts, best_inlinernum = \
                _RANSAC.RANSAM(points3D, Confidence_img, ransac_iteration = 50, inliner_threshold = 0.01)#1cm  0.003
                #not need find plane by RANSAC => for test not need RANSAC
                
                #turn bool img to uint8
                plane_img = plane_mask.astype(np.uint8)*255
                
                #Confi_mask : 去掉雜訊多的區域
                Confi_mask = np.zeros((171,224), np.uint8)
                cv2.circle(Confi_mask, (Confi_mask.shape[1]//2,Confi_mask.shape[0]//2), 110, 255 , -1)
                window_mask = _Feet_detection_v2.find_window_mask(window_corner, item_x.shape)
                feet_image, feet_top, feet_touch = _Feet_detection_v2.feet_detection(depthImg, window_mask, Confi_mask, height = 0.03, feet_height = 0.10, touch_detect_heigh = 0.1)
                print()
                if len(feet_touch) > 0:
                    _Tranform_Data.send_touch(feet_touch,points3D)
                    
                painter.paint (window_mask.astype(np.uint8)*255, 'mask')
                painter.paint (plane_img, 'plane_img')
                painter.paint (feet_image, 'feet_image')
        
        if(cv2.waitKey(10) & 0xFF == 113):#耗時0.01s
            break

In [ ]:
main()

Filename: feet.rrf
        Camera information
Type:            PICOFLEXX
Width:           224
Height:          171
Operation modes: 1
    MODE_PLAYBACK
        this operation mode has 1894904624 streams
Lens parameters: 9
    ('cx', 118.28559112548828)
    ('cy', 87.74105072021484)
    ('fx', 213.8031768798828)
    ('fy', 213.8031768798828)
    ('k1', 0.4155448377132416)
    ('k2', -4.7316107749938965)
    ('k3', 8.45906925201416)
    ('p1', 7.605663946304829e-16)
    ('p2', 4.939198934392371e-16)
CameraInfo items: 0
isConnected True
getFrameRate 0
UseCase MODE_PLAYBACK
  Quit : Q

create socket successful


































































































































































































































































































































































































